# Getting data from Public Health England
https://github.com/ScottishCovidResponse/scrc-vis-analytical/tree/master/ClusteringAndImpactAnalysis/data

In [8]:
import os
import sys
import json

module_path = os.path.abspath(os.pardir)
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pathlib import Path
from urllib import parse
import json
import requests

import pandas as pd

from stream import generate_streams, test_endpoints, get_token, register, generate_streams_from_urls

## Generate elements for urls.json

In [101]:
# uk_cumAdmissions = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumAdmissions&format=csv"
# uk_cumCasesBySpecimenDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumCasesBySpecimenDate&format=csv"
# uk_cumDeaths28DaysByDeathDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumDeaths28DaysByDeathDate&format=csv"
# uk_cumPeopleVaccinatedFirstDoseByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumPeopleVaccinatedFirstDoseByPublishDate&format=csv"
# uk_cumPeopleVaccinatedSecondDoseByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumPeopleVaccinatedSecondDoseByPublishDate&format=csv"
# uk_cumPeopleVaccinatedThirdInjectionByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumPeopleVaccinatedThirdInjectionByPublishDate&format=csv"
# uk_newAdmissions = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newAdmissions&format=csv"
# uk_newCasesBySpecimenDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newCasesBySpecimenDate&format=csv"
# uk_newDeaths28DaysByDeathDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newDeaths28DaysByDeathDate&format=csv"
# uk_newPeopleVaccinatedFirstDoseByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newPeopleVaccinatedFirstDoseByPublishDate&format=csv"
# uk_newPeopleVaccinatedSecondDoseByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newPeopleVaccinatedSecondDoseByPublishDate&format=csv"
# uk_newPeopleVaccinatedThirdInjectionByPublishDate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newPeopleVaccinatedThirdInjectionByPublishDate&format=csv"

# ltla_cases = "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=S12000036&metric=cumCasesBySpecimenDate&metric=newCasesBySpecimenDate&format=csv"
# ltla_deaths = "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=S12000036&metric=cumWeeklyNsoDeathsByRegDate&metric=newWeeklyNsoDeathsByRegDate&format=csv"
# ltla_vaccination = "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=S12000036&metric=cumVaccinationFirstDoseUptakeByVaccinationDatePercentage&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv"
# ltla_vaccinationAgeDemographics = "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=S12000036&metric=vaccinationsAgeDemographics&format=csv"

# msoa_newCasesBySpecimenDateRollingSum = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateRollingSum&format=csv"
# msoa_newCasesBySpecimenDateRollingRate = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateRollingRate&format=csv"
# msoa_newCasesBySpecimenDateChangePercentage = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateChangePercentage&format=csv"
# msoa_cumVaccinationFirstDoseUptakeByVaccinationDatePercentage = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=cumVaccinationFirstDoseUptakeByVaccinationDatePercentage&format=csv"
# msoa_cumVaccinationSecondDoseUptakeByVaccinationDatePercentage = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv"

In [6]:
urls = [
    "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateRollingSum&format=csv",
    "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateRollingRate&format=csv",
    "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=newCasesBySpecimenDateChangePercentage&format=csv",
    "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=cumVaccinationFirstDoseUptakeByVaccinationDatePercentage&format=csv",
    "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E02000961&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv"
]

In [7]:
template = {
    "name": "phe",
    "url": "https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=cumAdmissions&format=csv",
    "save_to": "phe/cumAdmissions.csv",
    "dataType": "cum_timeseries",
    "keywords": ["phe", "msoa", "hospital_admission", "daily"]
}

In [ ]:
for url in urls:
    d = template.copy()
    d['url'] = url
    d['save_to'] = f'phe/{dict(parse.parse_qsl(parse.urlsplit(url).query))["metric"]}.csv'
    print(json.dumps(d, indent=4))

## Data streams

In [10]:
with open('../manifest/urls.json') as f:
    urls = json.load(f)
    urls = [url for url in urls if url['name'] == 'phe']

streams = generate_streams_from_urls(urls)
print(streams[0])
len(streams)

{'urlCode': 'API_PY', 'endpoint': '/data/?product=phe/overview&component=cumAdmissions&format=long', 'dataType': 'cum_timeseries', 'keywords': ['phe', 'uk', 'hospital_admission', 'daily'], 'description': ''}


21

### UK

In [28]:
uk_streams = [s for s in streams if 'uk' in s['keywords']]
len(uk_streams)

12

### LTLA

In [5]:
ltla_streams = [s for s in streams if 'ltla' in s['keywords']]
len(ltla_streams)

4

Get all area codes, it can be done from API but only select the codes that have all data using downloaded data.

In [8]:
area_code_ltla_streams = []
n_expected_files = len(ltla_streams)
for p in Path('../../data/live/phe/ltla/').iterdir():
    if len(list(p.glob('*.csv'))) == n_expected_files:
        for s in ltla_streams:
            scopy = s.copy()
            scopy['endpoint'] = s['endpoint'].replace('phe/ltla', 'phe/ltla/' + p.name)
            scopy['keywords'] = scopy['keywords'] + [p.name]
            area_code_ltla_streams.append(scopy)
area_code_ltla_streams[0], len(area_code_ltla_streams)

({'urlCode': 'API_PY',
  'endpoint': '/data/?product=phe/ltla/s12000041&component=newCasesBySpecimenDate&format=long',
  'dataType': 'timeseries',
  'keywords': ['phe', 'ltla', 'new_cases', 'daily', 's12000041'],
  'description': ''},
 1344)

### MSOA

In [11]:
msoa_streams = [s for s in streams if 'msoa' in s['keywords']]
len(msoa_streams)

5

In [114]:
test_endpoints(area_code_ltla_streams[:100])

In [95]:
for s in area_code_ltla_streams:
    register(s, token, prod=True)

In [ ]:
# # To update title genration
# area_codes = requests.get('https://api.coronavirus.data.gov.uk/generic/area/msoa').json()
# area_codes_dict = {a['areaCode'].lower(): a['areaName'] for a in area_codes}
# print(json.dumps(area_codes_dict, indent=4))

---
## Playground

In [69]:
# age
age_case_url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=newCasesBySpecimenDateAgeDemographics&format=csv'
age_death_url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=newDeaths28DaysByDeathDateAgeDemographics&format=csv'
age_vaccine_url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=vaccinationsAgeDemographics&format=csv'

# new cases & deaths
case_death_url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=newCasesByPublishDate&metric=newDeaths28DaysByPublishDate&format=csv'

# vaccine
vaccine_url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=cumPeopleVaccinatedFirstDoseByVaccinationDate&metric=cumPeopleVaccinatedSecondDoseByVaccinationDate&metric=cumVaccinationFirstDoseUptakeByVaccinationDatePercentage&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv'

In [15]:
url_one = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=S12000036&metric=cumCasesBySpecimenDate&metric=newCasesBySpecimenDate&format=csv'
url_full = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&metric=cumCasesBySpecimenDate&metric=newCasesBySpecimenDate&format=csv'